In [1]:
import wget
import os
import subprocess
import sys
from subprocess import Popen
import pandas as pd
from shutil import which
import platform
import zipfile

### making gdc tool function

In [2]:
def download_tool(url):
    """Download tool using wget. Input is a URL."""
    filename = wget.download(url)
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall()
    return filename


In [4]:
def is_tool(name):
    """Check whether `name` is on PATH and marked as executable. Or if the exe file is in the directory."""
    check_1 = which(name) is not None
    files = os.listdir()
    filename = False
    for file in files:
        if "gdc-client" == file:
            filename = True
    return filename

In [5]:
op = platform.system()
tool_name = "gdc-client"
tool_avail = is_tool(tool_name)
print(tool_avail)
if tool_avail == False:
    """Check operating system and download appropriate gdc-client"""
    if op == "Darwin":
        download_tool('https://gdc.cancer.gov/files/public/file/gdc-client_v1.6.1_OSX_x64.zip')
#         download_tool('https://gdc.cancer.gov/files/public/file/gdc_dtt-ui_v1.0.0_mac_x64.zip')
    elif op == "Windows":
        download_tool('https://gdc.cancer.gov/files/public/file/gdc-client_v1.6.1_Windows_x64.zip')
#         download_tool('https://gdc.cancer.gov/files/public/file/gdc_dtt-ui_v1.0.0_win_x64.zip')
    elif op == "Linux":
        download_tool('https://gdc.cancer.gov/files/public/file/gdc-client_v1.6.1_Ubuntu_x64.zip')
#         download_tool('https://gdc.cancer.gov/files/public/file/gdc_dtt-ui_v1.0.0_linux_x86_64.AppImage.zip')


    


False
100% [........................................................................] 74322586 / 74322586

BadZipFile: File is not a zip file

In [ ]:
os.getcwd()

In [ ]:
# use the gdc client tool to download the files in your manifest
def use_gdc_tool(manifest_data):
    # create a new folder in the directory to store the files
    manifest_loc = "manifest_files"
    
    if os.path.isdir(manifest_loc):
        shutil.rmtree(manifest_loc)
    os.makedirs(manifest_loc)
    

    # changing directory path so files are downloaded into proper folder
    # need to change this path so it automatically changes paths to the folder (manifest_files)
    os.chdir(manifest_loc)

    # using the gdc tool to download files
    p1 = subprocess.run(['../gdc-client', 'download', '-m', manifest_data])
    return p1.stdout

#use_gdc_tool()

In [ ]:
use_gdc_tool('gdc_manifest.2023-05-18.txt')

###  need to access each sample file in each sample folder in manifest files and clean

In [ ]:
def get_clean_files():
    manifest_folders = []
    sample_filenames = []
    file_paths = []
    #access the folder with all of the downloaded file folders
    for folder in os.listdir('/Users/schw939/candle_stuff/GDC/manifest_files'):
        manifest_folders.append(folder)
    #if this exists in the list you created, get it out
    if '.DS_Store' in manifest_folders:
        manifest_folders.remove('.DS_Store')
    #get the names of each of the files that were downloaded    
    for folder_name in manifest_folders:
        for x in os.listdir('/Users/schw939/candle_stuff/GDC/manifest_files/' + folder_name):
            #get rid of the 'logs' folders that are contained in each folder
            if len(x) >4:
                sample_filenames.append(x)
    #remove checkpoint string if it exists
    for each_sample in sample_filenames:
        if '.ipynb_checkpoints' in each_sample:
            sample_filenames.remove('.ipynb_checkpoints')

    #set a list that will hold all of the cleaned up dataframes
    all_dataframes = []
    #get the final file path for each file by looping through each of the lists
    for folder_n, sample in zip(manifest_folders, sample_filenames):
        #read in all of the dataframes and add them to the df list
        dataframe = pd.read_csv((os.path.join('/Users/schw939/candle_stuff/GDC/manifest_files/' + folder_n + '/' + sample)), delimiter = '\t')  
        #clean up files so they read better
        dataframe = dataframe.reset_index()
        dataframe.columns = dataframe.iloc[0]
        #first rows of files don't look like they matter (tpm data is NAN)
        dataframe = dataframe[5:]
        #Grabbing the columns that we want
        if 'tpm_unstranded' in dataframe.columns:
            dataframe = dataframe[['gene_id', 'gene_name','gene_type', 'tpm_unstranded']]
            all_dataframes.append(dataframe)

        

    
    
    
    return all_dataframes

In [ ]:
get_clean_files()

# Mapping

In [ ]:
def map_and_combine(dataframe_list):
    df_list = []
    #read in genes.csv from cell_line data to map gene_name to entrez_id's
    genes = pd.read_csv('/Users/schw939/candle_stuff/data/genes.csv', delimiter = ',')
    #read in mutations.csv from cell_line data to map improve_id to entrez_id's
    mut = pd.read_csv('/Users/schw939/candle_stuff/data/mutations.csv', delimiter = ',')
   
    #for each dataframe map the entrez id using the gene_name column
    for df in dataframe_list:
        mapped_entrez = (df.merge(genes, left_on = 'gene_name', right_on = 'gene_symbol', how = 'left').reindex(columns = ['gene_id', 'gene_name', 'gene_type',
       'tpm_unstranded', 'entrez_id']))
    #for each dataframe map the improve_sample_id using the entrez_id column
        mapped_improve = (mapped_entrez.merge(mut, left_on = 'entrez_id', right_on = 'entrez_id', how = 'left').reindex(columns = ['gene_id', 'gene_name', 'gene_type', 
       'tpm_unstranded','entrez_id', 'improve_sample_id']))
    #merge all dataframes into one 
        df_list.append(mapped_improve)
    
    final_dataframe = pd. concat(df_list)

    return final_dataframe

In [ ]:
data = map_and_combine(get_clean_files())
# print(('there are {} missing values for entrez_id').format(data.entrez_id.isna().sum()))
# print(('there are {} missing values for improve_sample_id').format(data.improve_sample_id.isna().sum()))
# print(('there are {} missing values for tpm_unstranded').format(data.tpm_unstranded.isna().sum()))
data

In [ ]:
def clean_gene_id(final_dataframe):
    #getting rid of decimal places so this column can map to other id columns in datasets if needed
    other_id = []
    for id_name in final_dataframe.gene_id:
        if '.' not in id_name:
            other_id.append(id_name)
        else:
            head, sep, tail = id_name.partition('.')
            other_id.append(head)
            
    final_dataframe.gene_id = other_id
    return final_dataframe

In [ ]:
clean_gene_id(map_and_combine(get_clean_files()))